## 1 Introduction

This tutorial introduces the basic features for simulating titratable systems via the constant pH method.

We will consider a homogeneous aqueous solution of a titratable acidic species HA that can dissociate as follows
$HA \Leftrightarrow A^- + H^+\text{,}$
The constant pH method is a Monte Carlo mehtod.
If $N_0 = N_{\text{HA}} + N_{\text{A}^-}$ is the number of titratable groups in solution, then the degree of dissociation $\alpha$ can be defined:
$\alpha = \frac{N_{\text{A}^-}}{N_0}.$

In the constant pH method, the acceptance probability for a reaction is [1]:


$ P_{\mathrm{acc}} = \mathrm{min}\left(  1, \exp(\beta \Delta E_\mathrm{pot} \pm \ln{10} (\mathrm{pH - pK}) ) \right) $

and the proposal probability of a reaction is $P_\text{acc}=\frac{N_\text{HA}}{N0}$ for a dissociation and $P_\text{acc}=\frac{N_\text{A}}{N0}$ for an association reaction [1]. Here $\Delta E_\text{pot}$ is the potential energy change due to the reaction, while $\text{pH - p}K$ is an input parameter composed by two terms, pH and -p$K$.
The prefactor $\pm 1$ defines the direction of the reaction (+1 dissociation, -1 association).
When a dissociation move is attempted, the titratable molecule HA is charged and a counterion $H^+$ is randomly placed into the simulation box, while when an association move is attempted, a $A^-$ is neutralized and a random counterion $H^+$ is removed from the cell.
Be aware that in the constant pH method the system is coupled to an implicit pH bath. That especially means that the number of $H^+$ ions in the simulation box does not correspond to the chosen pH. This will lead to wrong screening effects when including electrostatic interactions in the system [1]. How to avoid these artifacts of the constant pH method, however, goes beyond the scope of this simple tutorial. We refer the reader to e.g. [1] for futher details.

## 2 Setup
We start by defining some important input parameters and setting the particles randomly inside the box:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.ion()

import espressomd
from espressomd import code_info
from espressomd import analyze
from espressomd import integrate
from espressomd import reaction_ensemble

# System parameters
#############################################################
box_l = 35

# Integration parameters
#############################################################
system = espressomd.System(box_l=[box_l] * 3)
system.set_random_state_PRNG()
#system.seed = system.cell_system.get_state()['n_nodes'] * [1234]
np.random.seed(seed=10)

system.time_step = 0.02
system.cell_system.skin = 0.4

# Particle setup
#############################################################
# type 0 = HA
# type 1 = A-
# type 2 = H+

N0 = 50  # number of titratable units
K = 1e-4


for i in range(N0):
    system.part.add(id=i, pos=np.random.random(3) * system.box_l, type=1)
for i in range(N0, 2 * N0):
    system.part.add(id=i, pos=np.random.random(3) * system.box_l, type=2)


The next step is to define the reaction system and seed the pseudo random number generator which is used for the Monte Carlo steps.
Please notice that the order in which the species are written in reactants and products lists is very important because, when a reaction is performed, the first species in the reactants list is replaced by the first species in the product lists, the second reactant species is replaced by the second product species, and so on. Moreover, if the reactant list has more species than the products list, reactant molecules in excess are deleted from the system, while if the products list has more species than the reactants list, product molecules in excess are created and randomly placed inside the simulation box. For example, reversing the order of products in our reaction (i.e. from  product_types=[type_H, type_A] to product_types=[type_A, type_H]), a neutral monomer would be positively charged and a negative monovalent counterion would be randomly placed inside the cell. 

In [ ]:
RE = reaction_ensemble.ConstantpHEnsemble(
        temperature=1, exclusion_radius=0, seed=77)


RE.add_reaction(gamma=K, reactant_types=[0], reactant_coefficients=[1],
                product_types=[1, 2], product_coefficients=[1, 1],
                default_charges={0: 0, 1: -1, 2: +1})
print(RE.get_status())

Next we perform simulations at different pH and make sure the system is equilibrated before taking samples

In [ ]:
num_samples=20

pHs=np.linspace(1,8,num=13)
degrees_of_dissociation=[]
std_errs_degree_of_dissociation=[]

for pH in pHs:
    print("pH is ", pH)
    RE.constant_pH = pH
    RE.reaction(2*N0) #warmup
    numAs=[]
    for i in range(num_samples):
        RE.reaction(3*N0)
        numAs.append(system.number_of_particles(type=1))
    degrees_of_dissociation.append(np.mean(numAs)/N0)
    std_errs_degree_of_dissociation.append(np.std(numAs,ddof=1)/(N0*np.sqrt(len(numAs))))

## 3 Results
Finally we plot our results and compare it to the well known result for an ideal reacting system. The errors in the simulation can be reduced via increasing num_samples taken at each pH.

In [ ]:
def ideal_degree_of_dissociation(pH, pK):
    return 1/(1+10**(pK-pH))

pK=-np.log10(K)
plt.figure(figsize=(10, 6), dpi=80)
plt.errorbar(pHs-pK, degrees_of_dissociation, std_errs_degree_of_dissociation, label=r"simulation")
loc_pHs=np.linspace(1,8)
plt.plot(loc_pHs-pK, ideal_degree_of_dissociation(loc_pHs, pK), label=r"ideal")
plt.xlabel('$pH-pK$', fontsize=20)
plt.ylabel(r'$\alpha$', fontsize=20)
plt.legend(fontsize=20)
plt.show()

## References

[1] Landsgesell, Jonas, Christian Holm, and Jens Smiatek. Simulation of weak polyelectrolytes: a comparison between the constant pH and the reaction ensemble method. The European Physical Journal Special Topics 226.4 (2017): 725-736.